In [1]:
# Mandatory cell for the rest of this assignment

%load_ext autoreload
%autoreload 2

from sys import path

path.append('../scripts')

In [2]:
# Création du dictionnaire d'NFT
dico_nft = {}

for i in range(0, 100):
    nftId = f"NFT{i}"
    dico_nft[nftId] = f"../nft_images/{nftId}.png"

In [3]:
# Import des modules nécessaires
from node import BlockchainNode
from network import Node
from wallet import Wallet
from proof_of_stake import ProofOfStake
from smart_contract import SmartContractDefinition, SmartContractWritingOperation

# Création de plusieurs wallets pour les nœuds de la blockchain
walletNode1 = Wallet()
walletNode2 = Wallet()
walletNode3 = Wallet()

# Création d'un wallet propriétaire pour le smart contract
walletProprietaire = Wallet()

# Création de wallets user :
walletMatthieu = Wallet()
walletEdouard = Wallet()
walletMilo = Wallet()
walletHugo = Wallet()
walletVincent = Wallet()
walletHenri = Wallet()

# Initialisation du mécanisme de Proof of Stake avec le premier nœud
pos = ProofOfStake(walletNode1.publicKey)

# On reset le network
Node.reset_network()

# On connecte les 3 noeuds au network
node1 = BlockchainNode(walletNode1,pos)
node2 = BlockchainNode(walletNode2,pos)
node3 = BlockchainNode(walletNode3,pos)

assert len(Node.get_all_nodes()) == 3
"Success"

'Success'

In [4]:
# Exemple de source de code pour le smart contract
source_code = """
class MyNFTContract:
    def __init__(self, issuerPublicKey):
        self.issuerPublicKey = issuerPublicKey
        self.nfts = {}  # Dictionnaire des NFTs

    def mint(self, ownerPublicKey, nftId):
        if nftId not in self.nfts:
            self.nfts[nftId] = ownerPublicKey
            print(f"NFT {nftId} frappé par {ownerPublicKey}")
        else:
            raise ValueError(f"NFT {nftId} déjà existant")

    def transfer(self, senderPublicKey, receiverPublicKey, nftId):
        if nftId in self.nfts and self.nfts[nftId] == senderPublicKey:
            self.nfts[nftId] = receiverPublicKey
            print(f"NFT {nftId} transféré de {senderPublicKey} à {receiverPublicKey}")
        else:
            raise ValueError("NFT inexistant ou propriété invalide")
"""

# Création du Smart Contract (définition)
NFT_contract = SmartContractDefinition(issuerPublicKey=walletProprietaire.publicKey, sourceCode=source_code, Ntoken=len(dico_nft))

# Signer le certificat
walletProprietaire.sign(NFT_contract)

# Instanciation du Smart Contract
NFT_instance = NFT_contract.instantiate_contract()

# Ajouter le Smart Contract au premier nœud (node1)
node1.new_certificate(NFT_contract) # Ajouter le smart contract au nœud

print(node1.blockchain.blockList)
print(node2.blockchain.blockList)
print(node3.blockchain.blockList)

Échec du forger du bloc. Diffusion du certificat : <smart_contract.SmartContractDefinition object at 0x0000019FDD7146E0>
Échec du forger du bloc. Diffusion du certificat : <smart_contract.SmartContractDefinition object at 0x0000019FDD7146E0>
Échec du forger du bloc. Diffusion du certificat : <smart_contract.SmartContractDefinition object at 0x0000019FDD7146E0>


In [5]:
# On vérifie si la période de minage est ouverte
NFT_contract.is_minting_open()

False

La période de minage est fermée donc les utilisateurs ne peuvent pas encore miner leur NFT. On peut quand même faire les tests :

In [6]:
try:
    assert NFT_contract.mint(ownerPublicKey=walletEdouard.publicKey, nftId="NFT1")
except ValueError as error:
    print(error)

La période de minage est fermée.


Il n'y a que le propriétaire du smart contract qui peut activer la période de minage. Ici c'est Alice !

In [7]:
NFT_contract.open_minting(walletProprietaire.publicKey, 1000)

Période de minage ouverte pendant 1000 secondes


In [8]:
assert NFT_contract.is_minting_open()
"Minting open!"

'Minting open!'

Maintenant qu'on a ouvert la periode de minage les utilisateurs peuvent miner des NFT

In [9]:
NFT_contract.mint(ownerPublicKey=walletEdouard.publicKey, nftId="NFT1")
print(NFT_contract.owners)

NFT NFT1 minée par 30820122300d06092a864886f70d01010105000382010f003082010a0282010100e2cbeccdfe597a82c61caa253794d7dbb13fd6cc735b21578b29a05381bf840cbb79eecc6a50f403bc6a6b84420407d9ae4681b697881b95bdf120081a7480126b76ad719f9ad3827bd6e2649c48e0f8c1cbf3aa43d4ec0d642386aceba2d9ec6efcbd063432ccfc0029c4edbe3da6b7ae418e316ccc1e3b49e8f51dcfbb5715639ed687c0570f2b9dd41b64f70bb39adfa0210f082134755a4cda8740b7b1bca09d9116b2eba6109dee95f32df6ee28dad86ab344362c9ea4807fbd682f35938fdb242ead46ce41efa479dcd7a95a1e6d9c38827ff3e1a095939fa7d2079e9950ebc0d28cd72a5305b72dc732a3a28359a33481bddf9016df6eaf1806f549790203010001
{'NFT1': '30820122300d06092a864886f70d01010105000382010f003082010a0282010100e2cbeccdfe597a82c61caa253794d7dbb13fd6cc735b21578b29a05381bf840cbb79eecc6a50f403bc6a6b84420407d9ae4681b697881b95bdf120081a7480126b76ad719f9ad3827bd6e2649c48e0f8c1cbf3aa43d4ec0d642386aceba2d9ec6efcbd063432ccfc0029c4edbe3da6b7ae418e316ccc1e3b49e8f51dcfbb5715639ed687c0570f2b9dd41b64f70bb39adfa0210f082134755a4cda8740b7

In [10]:
NFT_contract.mint(ownerPublicKey=walletMatthieu.publicKey, nftId="NFT2")
NFT_contract.mint(ownerPublicKey=walletHugo.publicKey, nftId="NFT3")
NFT_contract.mint(ownerPublicKey=walletMilo.publicKey, nftId="NFT4")
NFT_contract.mint(ownerPublicKey=walletHenri.publicKey, nftId="NFT5")

NFT NFT2 minée par 30820122300d06092a864886f70d01010105000382010f003082010a0282010100d73945456b7f21ad819277f129696912f93de0e82d07c0b5c0102fd98555b3f6ade20b1a0c7b35b9bcbddfd7dc35d5ee7d319e25af85a7cd8d7793432475c5e629ebe8ad8db5b5064d9ece0de7abcbaa1d8b0732fb94a2290e32e4294258a3ac12a91ce2923be8ae617a17f3ab6d503a615758917aaab16d7f9bce5ec93be331121727aa807d77a2dcda40b10711c78b1f6e6aafe208463959516e374a7b30f264f7f2de5fca7faa1bbd0f89e6c58bf6a011af1bd259cfad1caac1c6f23d929b475b058434e4a2369a43e8ee76df041c6c4678d3f288431f6587421a7d58eb7bdfc046774ae399ccde7cbae6c282888c468e443ad43c3ee24a148b8e351b13f50203010001
NFT NFT3 minée par 30820122300d06092a864886f70d01010105000382010f003082010a02820101009325d7906457c74d7cc01b6842cc2c04b4630ceef1dd837c66331a700fbbc336a61b5a3178399de69b6de36d5cd62e841af5a429748723b3d115cdab2b990bd7d94290e481e4db129b9b31227e3ba9310217c662974479a7b58aa617e7e258b7255afb8292ad5c13e62a2411cbb0fe1d9e10e4868ae64b3a926613e24f65f9bea2de1ea5d737e7e50ff8f0e96f6474cb00375cbe3f6094ab2d3

On peut changer le propriétaire d'un NFT si c'est le propriétaire de ce dernier qui le demande

In [12]:
NFT_contract.transfer(senderPublicKey=walletHenri.publicKey, receiverPublicKey=walletVincent.publicKey, nftId="NFT5")

NFT NFT5 transférée de 30820122300d06092a864886f70d01010105000382010f003082010a0282010100b151bff49a235224ee2f9c4036fc13a70e069d704a6e1a31cc9028e967fad660bcac613f86047e4c8a79f1854a6eeab4338448c16e2549abfd6dbc7bfbdd647decf9cff74e944cb5ecb3a83eaac0adcf2561de3338a22d342c48cddfdd8ce54192c93030d11147c5a0a0f71c1889b57a83589b866acadb2be60345033137d19af2ffa62924efc58e512255ca6580ddd0b9112c2bdb1813ae2b2d2c2576ad4c6d6357c27abb92c755d1888f600eb606bf65bdcd28e6873c4d9aad37cc29e163382a6a0c63b0f6e3cbcaf307567a219a0b56a07b5a0c20a3bd6a9635197b09cfe92d2e0c398b5741bd551b89c02d9ad442ec575cf71fafe57128d3ddbff37d4e610203010001 à 30820122300d06092a864886f70d01010105000382010f003082010a0282010100cbe83d19918bfaff4b5e1c471f24330b1378424e6669587255e71c545a1ace2387dda6e60f10a8e4dd1150a3e2f7be31a9ae3f96107bf9a82ddc04b79704b6abf6d6e71fd8940c418c7b65b8871a1ef18827ebf675d32b64f47fa8dba5c17ad40b27ef31f8eefcba89ac25254355dc2449a421564ed82fc0c7c7421808d7e15aac309895d54f8d69e019f30c8b1bcde5115c503e3cdc34bb8323a7dbd5b23481

In [13]:
NFT_contract.transfer(senderPublicKey=walletHenri.publicKey, receiverPublicKey=walletVincent.publicKey, nftId="NFT5")

ValueError: NFT n'appartient pas à l'expéditeur

In [ ]:
import matplotlib.pyplot as plt
from PIL import Image

# Affichage des 5 premières images avec leur ID
fig, axes = plt.subplots(1, 5, figsize=(15, 5))

for idx, (nftId, imagePath) in enumerate(dico_nft.items()):
    if idx >= 5:
        break
    try:
        image = Image.open(imagePath)
        axes[idx].imshow(image)
        axes[idx].set_title(nftId)
        axes[idx].axis('off')
    except FileNotFoundError:
        print(f"Image non trouvée : {imagePath}")
        axes[idx].set_title(f"{nftId}\n[Manquante]")
        axes[idx].axis('off')

plt.show()